In [328]:
import pandas as pd
import numpy as np
data=pd.read_csv('Train.csv')
data.head(5)

,Inv_Id,Vendor_Code,GL_Code,Inv_Amt,Item_Description,Product_Category
0,1,VENDOR-61,GL-6050100,6.973473,AETNA VARIABLE FUND - Apr-2002 - Store Managem...,CLASS-784
1,2,VENDOR-61,GL-6050100,25.053841,AETNA VARIABLE FUND - Nov-2000 - Store Managem...,CLASS-784
2,3,VENDOR-449,GL-6050100,53.573737,FAIRCHILD CORP - Nov-2001 - Store Management R...,CLASS-784
3,4,VENDOR-682,GL-6050100,67.388827,CALIFORNIA REAL ESTATE INVESTMENT TRUST - Aug-...,CLASS-784
4,5,VENDOR-682,GL-6050100,74.262047,CALIFORNIA REAL ESTATE INVESTMENT TRUST - Mar-...,CLASS-784


In [330]:
#Data analysis
print (data.GL_Code.nunique())
print (data.Vendor_Code.nunique())
print (data.Inv_Amt.nunique())
print (data.Item_Description.nunique())
print (data.shape)
data1=data.drop('Inv_Id',axis=1)   #Id is just the sequence, so droping it
#data1=data1.drop_duplicates(subset={'Vendor_Code','GL_Code'},keep='first',inplace=False) #droping dulicate entry if any
data1.head()
#data.isnull().sum() checking is any data entry is missing
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data1['Vendor_Code']=le.fit_transform(data1['Vendor_Code'])
data1['GL_Code']=le.fit_transform(data1['GL_Code'])
data1['Product_Category']=onhot.fit_transform(data1['Product_Category']) 
#data1['Product_Category']=onhot.fit_transform(data1['Product_Category'])  # for invererse Label encoder
print (data1.shape)

9
1313
5719
5118
(5719, 6)
(5719, 5)


In [331]:
# Feature Engineering
from nltk.corpus import stopwords
sw = stopwords.words("english")
import re
from nltk.stem.snowball import SnowballStemmer,PorterStemmer
import string
def clean_text(text):
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower().split()
    ## Remove stop words
    text = [w for w in text if not w in sw and len(w) >= 3]
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    txt = text.split()
    text = [i for i in txt if i.isalpha()]
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    text=text.lower()
    return text
descp=data1['Item_Description']
descp=descp.apply(clean_text)

In [332]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfvec=TfidfVectorizer(min_df=3, max_features=1000,strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)
transvec=tfvec.fit_transform(descp)
#transvec.shape
df=pd.DataFrame(transvec.toarray(), columns=tfvec.get_feature_names())
new_df=pd.concat([data1,df],axis=1)
new_df.drop('Item_Description',axis=1,inplace=True)

In [333]:
# Saving new modified data file after featured engineering
new_df.to_csv('Modified.csv')

In [334]:
#reading modified data file
mod_data=pd.read_csv('Modified.csv')
mod_data.drop(['Unnamed: 0'],axis=1,inplace=True)
X=mod_data.loc[:,mod_data.columns !='Product_Category']
Y=mod_data.loc[:,'Product_Category']
#splitting into train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25)
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=400,max_depth=40)
rfc.fit(x_train,y_train)
y_pred=rfc.predict(x_test)
x_test.shape

(1430, 1003)

In [335]:
x_test.shape

(1430, 1003)

In [336]:
pd.DataFrame(y_pred)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9986013986013986

In [337]:
#loading the test csv
test_df=pd.read_csv('Test.csv')
test_df_id=test_df['Inv_Id']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
test_df['Vendor_Code']=le.fit_transform(test_df['Vendor_Code'])
test_df['GL_Code']=le.fit_transform(test_df['GL_Code'])
test_df.drop('Inv_Id',axis=1,inplace=True)
test_descp=test_df['Item_Description']
test_descp=test_descp.apply(clean_text)  #applying cleaning text

In [338]:
from sklearn.feature_extraction.text import TfidfVectorizer
test_tfvec=TfidfVectorizer(min_df=3, max_features=1000,strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',ngram_range=(1, 3), use_idf=1, smooth_idf=1, sublinear_tf=1)
test_transvec=test_tfvec.fit_transform(test_descp)
df=pd.DataFrame(test_transvec.toarray(), columns=test_tfvec.get_feature_names())
test_df=pd.concat([test_df,df],axis=1)
test_df.drop('Item_Description',axis=1,inplace=True)
test_df.shape

(2292, 1003)

In [343]:
test_pred=rfc.predict(test_df)
test_pred_result=pd.DataFrame(test_pred)
test_pred_result.columns=['Product_Category']
test_pred_result.insert(0,'Inv_Id',test_df_id)
test_pred_result['Product_Category']=le.inverse_transform(test_pred_result['Product_Category'])
test_pred_result.to_csv('submission.csv',index=False)

In [344]:
test_pred_result.head()

,Inv_Id,Product_Category
0,6,CLASS-75
1,12,CLASS-75
2,14,CLASS-75
3,18,CLASS-75
4,19,CLASS-75


In [340]:
data['test']=le.fit_transform(data['Product_Category'])  ## For reversing the Labelencoder value
